In [26]:
import pandas as pd
import numpy  as np

Task 1: Create Import file (try, except)

In [27]:
print("Enter a class to grade (i.e. class1 for class1)")
name = input()
try:
    with open(name+'.txt','r') as gs:
        grades = gs.readlines()
        print("Successfully opened", name+'.txt')
        #print(grades)
except IOError:
        print("File not accessible")

Enter a class to grade (i.e. class1 for class1)
Successfully opened class2.txt


Task 2: Inspect Value (check if has 26 values, print invalid, report)

for loop to check if has 26 values, if not, print

In [28]:
new_grades_df = grades.copy()
for grade in new_grades_df: 
            value = grade.split(",")
            length = len(value)
            if length != 26:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " does not contain exactly 26 values:")
                print(grade)
                new_grades_df.remove(grade)
            id = value[0]
            id_length = len(id)
            if id_length != 9:
                corrupted_ID = value[0]
                print("Invalid line of data:", corrupted_ID, " is invalid")
                new_grades_df.remove(grade)
values_false = len(grades) - len(new_grades_df)
print("Total valid lines of data:", len(new_grades_df))
print("Total invalid lines of data:", values_false)


Invalid line of data: N00000023  does not contain exactly 26 values:
N00000023,,A,D,D,C,B,D,A,C,C,,C,,B,A,C,B,D,A,C,A,A

Invalid line of data: N0000002  is invalid
Invalid line of data: N00000035  does not contain exactly 26 values:
N00000035,B,A,D,D,B,B,,A,C,,D,B,A,B,A,A,B,D,A,C,A,C,B,D,D,A,A

Total valid lines of data: 22
Total invalid lines of data: 3


Task 3: Calculate Exam Test:
How many high score 
average score
highest score
lowest score 
range of score(highest minus lowest) 
question most people skip(question num - how many)
question most answer incorrectly(question num - how many)

----------
Student of highscore 
Median Score
Print score value
Get ratio of skip, incorrec

In [35]:
answer_keys = "B,A,D,D,C,B,D,A,C,C,D,B,A,B,A,C,B,D,A,C,A,A,B,D,D"
answer_key = answer_keys.split(",")
answer_key_table = pd.DataFrame(answer_key)
answers_checked = []
score_list = []
highScore_loop = []
for grade in new_grades_df:
        value = grade.split(",")
        id = value[0]
        #
        answer_table = pd.DataFrame(value)
        answer_table = answer_table.iloc[1: , :]
        answer_table = answer_table.reset_index(drop=True)
        #
        cVal = answer_table.iloc[24][0]
        cValSplit = cVal.split("\n")
        c = cValSplit[0]
        answer_table = answer_table.replace([cVal],c)
        #
        conditions  = [answer_table[0] == answer_key_table[0], answer_table[0] == "" , answer_table[0] != answer_key_table[0]]
        choices     = [ "True", '',answer_key_table[0]]
        answer_table["checkAnswer"] = np.select(conditions, choices, default=np.nan)
        #
        num_correct = (answer_table['checkAnswer']=='True').sum()
        num_blank = (answer_table['checkAnswer']=='').sum()
        num_incorrect = len(answer_table[answer_table['checkAnswer'].map(lambda x: x !='True' and x !='')])
        #
        score = num_correct*4 + num_incorrect*(-1)
        #
        score_df = pd.DataFrame({'id': [id], 'score': [score]})
        score_list.append(score_df)
        #
        highScore_list= score_df.loc[score_df['score'] > 80]

        highScore_loop.append(highScore_list)
        #print(highScore_list)
    


        answers_checked.append(answer_table)
        #print(answer_table)
#print(highScore_loop)
score_pd = pd.concat(score_list)
print(score_pd)

mean = score_pd['score'].mean()
max = score_pd['score'].max()
min = score_pd['score'].min()
RoS = max - min
print(RoS)
num_students = score_pd[id].nunique()

#sum score column and divide with num_students -> average


answer_pd = pd.concat(answers_checked)
answer_pd.columns = ['studentAnswer', 'checkAnswer']
answer_pd['question'] = answer_pd.index

#Find out the question students most answer incorrectly and how many time the question is incorrect.
answers_incorrect_pd = answer_pd[answer_pd['checkAnswer'] != "True"]
answers_incorrect_pd = answers_incorrect_pd[answers_incorrect_pd['studentAnswer'] != ""]
answer_incorrect_count = answers_incorrect_pd.groupby('question')['question'].count()
question_most_incorrect = answer_incorrect_count.idxmax()
num_answer_inccorect = answer_incorrect_count.max()
print("Question that most people answer incorrectly:", question_most_incorrect)
print("Question", question_most_incorrect, "is answered incorrectly", num_answer_inccorect, "times")
print("----------------------")

#Find out the question students most leave blank and how many time the question is leave blank.
answers_blank_pd = answer_pd[answer_pd['studentAnswer'] == ""]
answers_blank_count = answers_blank_pd.groupby('question')['question'].count()
question_most_leave_blank = answers_blank_count.idxmax()
num_answer_blank = answers_blank_count.max()
print("Question that most people skip:", question_most_leave_blank)
print("Question", question_most_leave_blank, "is skipped", num_answer_blank, "times")






          id  score
0  N00000021     68
0  N00000022     76
0  N00000024     73
0  N00000026     72
0  NA0000027     79
0  N00000028     73
0  N00000029     87
0  N00000030     82
0  N00000031     76
0  N00000032     87
0  N00000033     77
0  N00000034     69
0  N00000036     77
0  N00000037     75
0  N00000038     73
0  N00000039     66
0  N00000040     73
0  N00000041     91
0  N00000042    100
0  N00000043     86
0  N00000044     90
0  N00000045     67
34


KeyError: 'N00000045'

Task 4: Export text file: 
as xxx_grades.txt
list as 'id', 'score'